In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
SKELETON_DIR = '/content/drive/MyDrive/DLE602'

# Set up auto-reloading modules from the working directory
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# Set the default figure size
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 120

In [15]:
%%bash
curl -L -o ./lung-x-ray-image-clinical-text-dataset.zip\
  https://www.kaggle.com/api/v1/datasets/download/ghostbat101/lung-x-ray-image-clinical-text-dataset

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1358M  100 1358M    0     0   101M      0  0:00:13  0:00:13 --:--:--  171M


In [18]:
%%bash
rm -rf dataset
mkdir dataset
unzip -q -o lung-x-ray-image-clinical-text-dataset.zip -d dataset/